# **Redes Quânticas - Teletransporte Quântico em uma Rede Quântica com Erros**

Neste notebook, por último, introduzimos erros para a memória e tempos de atraso para os canais no sistema. Isto permite-nos investigar como a fidelidade do estado teletransportado muda sob condições mais “realistas”.

In [7]:
import netsquid as ns
import pandas
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.nodes import Node, Connection, Network
from netsquid.protocols.protocol import Signals
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.components.qchannel import QuantumChannel
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components.qprogram import QuantumProgram
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel
from netsquid.components.models.delaymodels import FibreDelayModel, FixedDelayModel
import netsquid.components.instructions as instr
import pydynaa

<br><br>Primeiramente, introduzimos ruído no processador e simulamos que qubits em qualquer posição de memória experimentam ruído de despolarização quando ociosos.

Também estendemos os tempos de duração das portas (para ver melhor o efeito dos ruídos).

In [2]:
phys_instructions3 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_S, duration=10, parallel = True, topology =[0]),
    PhysicalInstruction(instr.INSTR_H, duration=10, prallel = True, topology = [0,1]),
    PhysicalInstruction(instr.INSTR_X, duration=10, parallel = True, topology = [0]),
    PhysicalInstruction(instr.INSTR_Z, duration=10, parallel = True, topology = [0]),
    PhysicalInstruction(instr.INSTR_CNOT, duration=1, parallel = True, topology = [(0,1),(1,2)]),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel = True, topology = [0,1])
]

qproc3a = QuantumProcessor(name="ExampleQProc3a", num_positions = 3,
                           mem_noise_models=[DepolarNoiseModel(1e12)]*3, 
                           phys_instructions = phys_instructions3)

qproc3b = QuantumProcessor(name="ExampleQProc3b", num_positions = 3,
                           mem_noise_models=[DepolarNoiseModel(1e7)]*3, 
                           phys_instructions = phys_instructions3)

<br><br>Também levamos em consideração quanto tempo leva para a informação viajar pela fibra. Para isso, primeiro incluímos um modelo de atraso no canal quântico e no canal clássico.

In [3]:
class QuantumConnection3(Connection):
    """Uma conexão que neste exemplo contém apenas um canal quântico. 
    No entanto, podemos incluir uma fonte quântica desde a conexão
    e usá-la para distribuir qubits emaranhados aos nós conectados."""
    def __init__(self, length, name = "QuantumConnection"):
        super().__init__(name=name)
        
        # Inicializando um canal quântico
        qchannel3 = QuantumChannel(name="QChannel_3a23b", length = length,
                                  models={"delay_model":FibreDelayModel()})
        # Conectando o canal as portas de entrada e saídas da conexão
        self.add_subcomponent(qchannel3, forward_input=[("A","send")],forward_output=[("B", "recv")])
        
class ClassicalConnection3(Connection):
    """Uma conexão com um canal clássico"""
    def __init__(self,length, name="ClassicalConnection"):
        super().__init__(name=name)

        # Inicializando o canal clássico
        cchannel3 = ClassicalChannel(name="QChannel_3a23b",
                                  length = length,
                                  models={"delay_model":FibreDelayModel()})

        # Conectando o canal com as portas de entrada e saída da conexão
        self.add_subcomponent(cchannel3, 
                             forward_input=[("A","send")],
                             forward_output=[("B","recv")])

In [4]:
def network_setup3(length_channel = 8e-3):
    """Configurando todos os componentes da rede"""
    
    # Primeiro nós configuramos os dois nós com os seus processadores
    
    node3a = Node(name="ExampleNode3a", qmemory = qproc3a, port_names=['cin_3a','cout_3a','qin_3a','qout_3a'])
    node3b = Node(name="ExampleNode3b", qmemory = qproc3b, port_names=['cin_3b','cout_3b','qin_3b','qout_3b'])
    # Em seguida criamos a rede
    network3 = Network("ExampleNet3")
    
    # Agora, nós adicionamos os dois nós na rede
    network3.add_nodes([node3a,node3b])
    
    # Configurando a conexão clássica entre os dois nós
    cconn = ClassicalConnection3(length=length_channel)
    
    # E adicionando as conexões na rede
    network3.add_connection(node3a, node3b, connection=cconn, label = "classical connection",
                           port_name_node1="cout_3a",port_name_node2="cin_3b")
    # Configurando uma conexão quântica entre os nós
    qconn = QuantumConnection3(length=length_channel)
    
    # Adicionando a conexão na rede
    network3.add_connection(node3a, node3b, connection=qconn, label = "quantum connection",
                           port_name_node1="qout_3a",port_name_node2="qin_3b")
    
    # por último, adicionamos conexões as portas dentro de cada nó
    node3a.qmemory.ports['qout2'].forward_output(node3a.ports['qout_3a'])
    node3b.ports['qin_3b'].forward_input(node3b.qmemory.ports['qin0'])
    
    return network3

In [5]:
class node3aProgram1(QuantumProgram):
    default_num_qubits = 3 # número de qubits
    
    def program(self):
        q0, q1, q2 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_INIT, [q0,q1,q2]) # inicializando os três qubits
        
        # Configurando o estado do qubit 1 para y0
        self.apply(instr.INSTR_H, [q0]) # Aplicando a porta H em q0
        self.apply(instr.INSTR_S, [q0]) # Aplicando a porta S em q0
        
        # Emaranhando os qubits 2 e 3
        self.apply(instr.INSTR_H, [q1]) # Aplicando a porta H em q1
        self.apply(instr.INSTR_CNOT, [q1,q2]) # Aplicando a porta CNOT em q1 como controle e q2 como alvo
        
        yield self.run() # Rodando as instruções acima
    
class node3aProgram2(QuantumProgram):
    default_num_qubits = 2 # Número de qubits
    
    def program(self):
        q0, q1 = self.get_qubit_indices(2)
        
        # Agora emaranhando os qubits 1 e 2 e medindo ambos 
        self.apply(instr.INSTR_CNOT, [q0,q1]) # Aplicando a porta CNOT em q0 como controle e q1 como alvo
        self.apply(instr.INSTR_H, [q0]) # Aplicando a porta H em q0
        self.apply(instr.INSTR_MEASURE, q0, output_key = "M1") # medir o q0 e armazena o resultado
        self.apply(instr.INSTR_MEASURE, q1, output_key = "M2") # medir o q1 e armazena o resultado
        
        yield self.run() # Rodando as instruções agendadas acima

class node3aProtocol(NodeProtocol):
    def run(self):
        node3aprog1 = node3aProgram1()
        node3aprog2 = node3aProgram2()
        
        yield self.node.qmemory.execute_program(node3aprog1)
        
        qubit = self.node.qmemory.pop(positions=[2])
        
        self.node.qmemory.ports["qout2"].tx_output(qubit)
        
        yield self.node.qmemory.execute_program(node3aprog2)
        
        m1, = node3aprog2.output["M1"]
        m2, = node3aprog2.output["M2"]
        print("measurement outcome", m1, m2)
        self.node.ports["cout_3a"].tx_output((m1,m2))
        
        
class node3bProtocol(NodeProtocol):
    def run(self):
        port_cin = self.node.ports["cin_3b"]
        port_qin = self.node.ports["qin_3b"]
        
        # Aguardando o qubit chegar aqui
        expr = yield(self.await_port_input(port_qin))
        
        
        # Aguardando a mensagem chegar aqui
        expr = yield(self.await_port_input(port_cin))
        message, = self.node.ports["cin_3b"].rx_input().items
        print("message", message)
        if message[0] == 1:
            self.node.qmemory.execute_instruction(instr.INSTR_X,[0]) # Aplicando a porta H
            print("aplicando porta X")
            yield self.await_program(self.node.qmemory)
        
        if message[1] == 1:
            self.node.qmemory.execute_instruction(instr.INSTR_Z,[0]) # Aplicando a porta Z
            print("aplicando porta Z")
            yield self.await_program(self.node.qmemory)

In [6]:
ns.set_random_state(seed=47) # definir "sementes" para garantir um estado despolarizado na primeira execução

network = network_setup3()

node3a = network.get_node("ExampleNode3a")
node3b = network.get_node("ExampleNode3b")

node3aprot = node3aProtocol(node3a)
node3bprot = node3bProtocol(node3b)

node3aprot.start() #iniciando o protocolo no node3a
node3bprot.start() #iniciando o protocolo no node3b

ns.sim_run()
ns.sim_time()

qubit, = node3b.qmemory.peek(positions=[0])
print("Fidelity of teleported state:{}".format(ns.qubits.fidelity([qubit],ns.y0)))

print("Teleported state", qubit.qstate.qrepr)

print("goal state", ns.y0)

measurement outcome 1 1
message (1, 1)
aplicando porta X
aplicando porta Z
Fidelity of teleported state:0.0
Teleported state KetRepr(num_qubits=1,
ket=
[[ 7.07106781e-01-1.73191211e-16j]
 [-1.73191211e-16-7.07106781e-01j]])
goal state [[0.70710678+0.j        ]
 [0.        +0.70710678j]]
